In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import logging
import ast
from shapely.geometry import Point
import matplotlib.pyplot as plt
from typing import Dict, List, Optional, Tuple
import re

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [2]:
# Configuration
YEAR = 1965
SIDE = "west"
DATA_DIR = Path('../data')
RAW_DIR = DATA_DIR / 'raw' / SIDE
INTERIM_DIR = DATA_DIR / 'interim'
PROCESSED_DIR = DATA_DIR / 'processed'
GEO_DATA_DIR = DATA_DIR / 'data-external'

In [3]:
# U-Bahn line profile mappings
KLEINPROFIL = {'1', '2', '3', '4', 'A', 'A I', 'A II', 'A III', 'A1', 'A2', 'B', 'B I', 'B II', 'B III', 'B1', 'B2'}
GROSSPROFIL = {'5', '6', '7', '8', '9', 'C', 'C I', 'C II', 'D', 'E', 'G'}

In [4]:
# Load stations
matched_path = f"../data/interim/stops_base/lines_{YEAR}_{SIDE}.csv"
line_df_initial = pd.read_csv(matched_path)

In [5]:
def enrich_lines(line_df):
    # Add profile for U-Bahn lines
    line_df['profile'] = None
    for idx, row in line_df.iterrows():
        if row['type'] == 'u-bahn':
            if row['line_name'] in KLEINPROFIL:
                line_df.at[idx, 'profile'] = 'Kleinprofil'
            elif row['line_name'] in GROSSPROFIL:
                line_df.at[idx, 'profile'] = 'Großprofil'
    
    # Add capacity based on transport type and profile
    line_df['capacity'] = None
    for idx, row in line_df.iterrows():
        if row['type'] == 'u-bahn' and row['profile'] == 'Kleinprofil':
            line_df.at[idx, 'capacity'] = 750
        elif row['type'] == 'u-bahn' and row['profile'] == 'Großprofil':
            line_df.at[idx, 'capacity'] = 1000
        elif row['type'] == 's-bahn':
            line_df.at[idx, 'capacity'] = 1100
        elif row['type'] == 'tram':
            line_df.at[idx, 'capacity'] = 195
        elif row['type'] in ['autobus', 'bus (Umlandlinie)']:
            line_df.at[idx, 'capacity'] = 100
        elif row['type'] == 'fähre' or row['type'] == 'FÃ¤hre':
            line_df.at[idx, 'capacity'] = 300
    
    return line_df

line_df = enrich_lines(line_df_initial)


In [6]:
# Load stations
matched_path = f"../data/interim/stops_verified/stops_{YEAR}_{SIDE}.csv"
final_stops = pd.read_csv(matched_path)

def load_district_data():
    """Load district boundary data"""
    try:
        districts_path = GEO_DATA_DIR / "lor_ortsteile.geojson"
        districts_gdf = gpd.read_file(districts_path)
        
        # Load West Berlin districts
        west_berlin_path = GEO_DATA_DIR / "West-Berlin-Ortsteile.json"
        with open(west_berlin_path, "r") as f:
            west_berlin_districts = json.load(f)["West_Berlin"]
            
        return districts_gdf, west_berlin_districts
    except Exception as e:
        logger.error(f"Error loading district data: {str(e)}")
        return None, None

def add_administrative_data(stops_df, districts_gdf, west_berlin_districts):
    """Add administrative boundary information to stops"""
    # Make a copy to avoid modifying the original
    result_df = stops_df.copy()
    
    # Filter to only stops with location data
    stops_with_location = result_df[result_df['location'].notna()]
    
    if stops_with_location.empty:
        logger.warning("No stops with location data found, skipping administrative enrichment")
        return result_df
    
    try:
        # Convert to GeoDataFrame
        geometry = stops_with_location['location'].apply(
            lambda x: Point(*reversed([float(c.strip()) for c in x.split(',')]))
        )
        stops_gdf = gpd.GeoDataFrame(stops_with_location, geometry=geometry, crs="EPSG:4326")
        
        # Ensure CRS matches
        if stops_gdf.crs != districts_gdf.crs:
            districts_gdf = districts_gdf.to_crs(stops_gdf.crs)
        
        # Perform spatial join
        joined = gpd.sjoin(
            stops_gdf, 
            districts_gdf[['geometry', 'BEZIRK', 'OTEIL']], 
            how="left", 
            predicate='within'
        )
        
        # Add east/west classification
        joined['east_west_admin'] = joined['OTEIL'].apply(
            lambda x: 'west' if pd.notna(x) and x in west_berlin_districts else 'east'
        )
        
        # Drop geometry column and index_right from join
        if 'geometry' in joined.columns:
            joined = joined.drop(columns=['geometry'])
        if 'index_right' in joined.columns:
            joined = joined.drop(columns=['index_right'])
        
        # Update original DataFrame
        for idx, row in joined.iterrows():
            original_idx = stops_df[stops_df['stop_id'] == row['stop_id']].index[0]
            result_df.at[original_idx, 'district'] = row.get('BEZIRK')
            result_df.at[original_idx, 'neighborhood'] = row.get('OTEIL')
            result_df.at[original_idx, 'east_west_admin'] = row.get('east_west_admin')
        
        # Validate east/west classification
        for idx, row in result_df.iterrows():
            if pd.notna(row.get('east_west_admin')) and pd.notna(row.get('east_west')):
                if row['east_west_admin'] != row['east_west']:
                    logger.warning(f"Mismatched east/west: {row['stop_name']} - Source says {row['east_west']} but location is in {row['east_west_admin']}")
        
        return result_df
    except Exception as e:
        logger.error(f"Error adding administrative data: {str(e)}")
        return result_df

# Load district data
districts_gdf, west_berlin_districts = load_district_data()

# Add administrative data
if districts_gdf is not None and west_berlin_districts is not None:
    enriched_stops_df = add_administrative_data(final_stops, districts_gdf, west_berlin_districts)
    
    # Save the enriched stops
    (INTERIM_DIR / 'stops_enriched').mkdir(exist_ok=True)
    enriched_stops_df.to_csv(INTERIM_DIR / 'stops_enriched' / f'stops_{YEAR}_{SIDE}_enriched.csv', index=False)
    
    logger.info(f"Enriched stops saved to interim/stops_enriched directory")
else:
    logger.warning("Could not load district data, skipping administrative enrichment")
    enriched_stops_df = final_stops

2025-02-28 20:56:14,764 - INFO - Enriched stops saved to interim/stops_enriched directory


In [8]:
import requests
import geojson


# Specify the url for the backend.
url = "https://gdi.berlin.de/services/wfs/postleitzahlen"

# Specify parameters (read data in json format).
params = dict(
    service="WFS",
    version="2.0.0",
    request="GetFeature",
    typeNames="postleitzahlen",
    outputFormat="json",
)

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson and set coordinate reference system
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content), crs="EPSG:25833")

# Reproject data_gdf to EPSG:4326
data = data.to_crs(epsg=4326)

# Convert final_stops to GeoDataFrame
geometry = final_stops['location'].apply(lambda x: Point(*reversed([float(c.strip()) for c in x.split(',')])))
final_stops_gdf = gpd.GeoDataFrame(final_stops, geometry=geometry, crs="EPSG:4326")

# Perform spatial join to find which postal code each stop is in
stops_with_plz = gpd.sjoin(final_stops_gdf, data, how='left', predicate='within')
# Add the 'plz' column to the original stops_df
final_stops['plz'] = stops_with_plz['plz']

In [10]:
# import raw
matched_path = f"../data/raw/{YEAR}_{SIDE}.csv"
base_df = pd.read_csv(matched_path)

def create_line_stops_df(df):
    line_stops = df['stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

line_stops = create_line_stops_df(base_df)

In [11]:
# Increment the index by 1 and set it as 'line_id'
line_stops['line_id'] = str(YEAR) + (line_stops["index"] + 1).astype(str)
# Drop the 'index' column
line_stops.drop(columns=['index'], inplace=True)

In [12]:
def add_type(line_stops, line_df):
    # Ensure line_id columns are of the same type
    line_stops['line_id'] = line_stops['line_id'].astype(str)
    line_df['line_id'] = line_df['line_id'].astype(str)
    
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

line_stops = add_type(line_stops, line_df)

In [13]:
def add_fk(line_stops, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type']]

    # Merge the line_stops and stop_id_df dataframes based on matching stop names and type
    line_stops = line_stops.merge(stop_id_df,
                                  left_on=['stop_name', 'type'],
                                  right_on=['stop_name', 'type'],
                                  how='left')

    return line_stops

# Assuming line_stops and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'line_name' with the actual column names you have

line_stops = add_fk(line_stops, final_stops)


In [14]:
# Calculate the difference between consecutive 'stop_order' values
line_stops['diff'] = line_stops['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops = line_stops[line_stops['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops[(line_stops['diff'] != 1) & (line_stops['stop_order'] != 0)]
faulty_rows

,stop_name,stop_order,line_id,type,line_name,stop_id,diff


In [15]:
line_stops

,stop_name,stop_order,line_id,type,line_name,stop_id,diff
0,"Marienfelde, Daimlerstrasse",0,19651,tram,15,19650,NaN
1,Großbeerenstrasse Ecke Daimlerstrasse,1,19651,tram,15,19651,1.0
2,Körtingstrasse Ecke Großbeerenstrasse,2,19651,tram,15,19652,1.0
3,Mariendorferdamm Ecke Alt-Mariendorf,3,19651,tram,15,19653,1.0
4,Imbrosweg Ecke Rixdorferstrasse,4,19651,tram,15,19654,1.0
...,...,...,...,...,...,...,...
1434,Turmstrasse,4,1965102,u-bahn,G,19651004,1.0
1435,Hansaplatz,5,1965102,u-bahn,G,19651005,1.0
1436,Zoologischer Garten,6,1965102,u-bahn,G,1965941,1.0
1437,Kurfürstendamm,7,1965102,u-bahn,G,1965965,1.0


In [16]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops['stop_id'] = pd.to_numeric(line_stops['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops[line_stops['stop_id'].isnull()])

line_stops.drop(['line_name', "stop_name", "type", "diff"], axis=1, inplace=True)

All values in 'stop_id' column are numeric


In [17]:
def finalize_data(line_df, stops_df, line_stops):
    """Prepare final datasets for output"""
    # Make copies to avoid modifying originals
    final_line_df = line_df.copy()
    final_stops_df = stops_df.copy()
    final_line_stops_df = line_stops.copy()
    
    # Ensure consistent column names and formats
    
    # For lines table
    if 'profile' not in final_line_df.columns:
        final_line_df['profile'] = None
    
    # Sort tables
    final_line_df = final_line_df.sort_values('line_id')
    final_stops_df = final_stops_df.sort_values('stop_id')
    final_line_stops_df = final_line_stops_df.sort_values(['line_id', 'stop_order'])
    
    return final_line_df, final_stops_df, final_line_stops_df

# Finalize data
final_line_df, final_stops_df, final_line_stops_df = finalize_data(line_df, enriched_stops_df, line_stops)

# Save final data
(PROCESSED_DIR / str(YEAR)).mkdir(parents=True, exist_ok=True)
final_line_df.to_csv(PROCESSED_DIR / str(YEAR) / f'lines_{YEAR}_{SIDE}.csv', index=False)
final_stops_df.to_csv(PROCESSED_DIR / str(YEAR) / f'stops_{YEAR}_{SIDE}.csv', index=False)
final_line_stops_df.to_csv(PROCESSED_DIR / str(YEAR) / f'line_stops_{YEAR}_{SIDE}.csv', index=False)

logger.info(f"Final data saved to processed directory")

2025-02-28 20:57:07,972 - INFO - Final data saved to processed directory


In [18]:
# =============================================
# STAGE 8: UPDATE REFERENCE DATA
# =============================================

# load existing stations
existing_stations_df = pd.read_csv("../data/processed/existing_stations.csv")

logger.info("STAGE 8: Updating reference data")

def update_reference_data(new_stops_df, existing_stations_df):
    """Update the reference stations dataset with new geolocated stops"""
    # Make a copy of existing stations
    updated_stations = existing_stations_df.copy() if existing_stations_df is not None else pd.DataFrame()
    
    # Filter to stops with location
    new_stops_with_location = new_stops_df[new_stops_df['location'].notna()]
    
    # Create a set of existing stop identifiers (stop_name + type)
    if not updated_stations.empty:
        existing_identifiers = set(
            updated_stations.apply(lambda x: f"{x['stop_name']}|{x['type']}", axis=1)
        )
    else:
        existing_identifiers = set()
    
    # Add new stops that aren't in the existing stations
    new_stops = []
    updated_count = 0
    
    for _, stop in new_stops_with_location.iterrows():
        stop_identifier = f"{stop['stop_name']}|{stop['type']}"
        
        if stop_identifier not in existing_identifiers:
            # This is a new stop, add it
            new_stops.append(stop)
        else:
            # This is an existing stop, update location if it was missing
            matching_idx = updated_stations[
                (updated_stations['stop_name'] == stop['stop_name']) &
                (updated_stations['type'] == stop['type'])
            ].index
            
            if not matching_idx.empty:
                idx = matching_idx[0]
                if pd.isna(updated_stations.at[idx, 'location']):
                    updated_stations.at[idx, 'location'] = stop['location']
                    updated_count += 1
    
    # Convert new stops to DataFrame
    if new_stops:
        new_stops_df = pd.DataFrame(new_stops)
        
        # Ensure consistent columns with existing_stations
        if not updated_stations.empty:
            for col in updated_stations.columns:
                if col not in new_stops_df.columns:
                    new_stops_df[col] = None
            
            # Select only the columns in existing_stations
            new_stops_df = new_stops_df[updated_stations.columns]
        
        # Combine with existing stations
        updated_stations = pd.concat([updated_stations, new_stops_df], ignore_index=True)
    
    logger.info(f"Added {len(new_stops)} new stops to reference data")
    logger.info(f"Updated locations for {updated_count} existing stops")
    
    return updated_stations

# Update reference data
updated_reference = update_reference_data(final_stops_df, existing_stations_df)

# Save updated reference data
updated_reference.to_csv(PROCESSED_DIR / 'existing_stations.csv', index=False)

logger.info(f"Updated reference data saved to processed directory")

2025-02-28 20:57:08,028 - INFO - STAGE 8: Updating reference data


2025-02-28 20:57:08,526 - INFO - Added 0 new stops to reference data
2025-02-28 20:57:08,527 - INFO - Updated locations for 0 existing stops
2025-02-28 20:57:08,541 - INFO - Updated reference data saved to processed directory


In [19]:
logger.info("Processing complete")

print("\n" + "="*50)
print(f"SUMMARY: {YEAR} {SIDE.upper()} BERLIN PROCESSING")
print("="*50)
print(f"Lines processed: {len(final_line_df)}")
print(f"Stops processed: {len(final_stops_df)}")
print(f"Line-stop relationships processed: {len(final_line_stops_df)}")
print(f"Stops with location data: {final_stops_df['location'].notna().sum()} ({final_stops_df['location'].notna().sum()/len(final_stops_df)*100:.1f}%)")

if 'district' in final_stops_df.columns:
    print(f"Stops with district data: {final_stops_df['district'].notna().sum()} ({final_stops_df['district'].notna().sum()/len(final_stops_df)*100:.1f}%)")

print("\nFiles created:")
print(f"- {PROCESSED_DIR / f'lines_{YEAR}_{SIDE}.csv'}")
print(f"- {PROCESSED_DIR / f'stops_{YEAR}_{SIDE}.csv'}")
print(f"- {PROCESSED_DIR / f'line_stops_{YEAR}_{SIDE}.csv'}")
print(f"- {PROCESSED_DIR / 'existing_stations.csv'} (updated reference data)")

print("\nNext steps:")
print("1. Review and manually correct any stops missing location data")
print("2. Process East Berlin 1965 data using the same workflow")
print("3. Compare West and East Berlin networks using visualization tools")
print("4. Proceed to temporal analysis by processing adjacent years (1964, 1966)")
print("="*50)

2025-02-28 20:57:08,562 - INFO - Processing complete



SUMMARY: 1965 WEST BERLIN PROCESSING
Lines processed: 102
Stops processed: 1066
Line-stop relationships processed: 1396
Stops with location data: 1066 (100.0%)
Stops with district data: 1010 (94.7%)

Files created:
- ..\data\processed\lines_1965_west.csv
- ..\data\processed\stops_1965_west.csv
- ..\data\processed\line_stops_1965_west.csv
- ..\data\processed\existing_stations.csv (updated reference data)

Next steps:
1. Review and manually correct any stops missing location data
2. Process East Berlin 1965 data using the same workflow
3. Compare West and East Berlin networks using visualization tools
4. Proceed to temporal analysis by processing adjacent years (1964, 1966)
